In [8]:
import numpy as np
import pandas as pd


In [9]:
# load data into df
volumes_df = pd.read_csv('origination_volumes.csv')
rates_df = pd.read_csv('loss_rates.csv')

In [10]:
# merge reporting date and portfolio to a new columne 
volumes_df ['Index'] = volumes_df ['Reporting_Date'].astype(str) + volumes_df ['Portfolio']
rates_df ['Index'] = rates_df ['reporting_date'].astype(str) + rates_df  ['portfolio']

In [11]:
# Join two tables by left join 
predicted_loss_df = pd.merge(volumes_df, rates_df, how = "left")

# leave to user if dropna is preferred 
# predicted_loss_df.dropna()

In [12]:
# calculate predicted loss = loan value * expected lose rate 
predicted_loss_df ['predicted_losses'] = predicted_loss_df ['Total_Loan_Value_Originated'] * predicted_loss_df ['expected_losses'] 

In [39]:
#filter 2016 data 
predicted_loss_df ['filter'] = predicted_loss_df['Index'].str.contains('2016', case=False, na='1')

#predicted_loss_df.drop('filter')

# sum predicted loss = total predicted loss in 2016
sum_loss_df= predicted_loss_df.groupby('filter')['predicted_losses'].sum()

# rename index to non-target and 2016
per_year= sum_loss_df.rename(index={False: 'Non-Target Years',True: '2016'})


,Reporting_Date,Portfolio,Lender_Type,Insurance_Status,Dimension,Category,Total_Loan_Value_Originated,Index,reporting_date,portfolio,expected_losses,predicted_losses,filter
0,2022-03-31,mtg,Chartered Bank,Insured,Credit Score Range,0 < 599,43478000.0,2022-03-31mtg,NaN,NaN,NaN,NaN,False
1,2022-03-31,mtg,Chartered Bank,Insured,Credit Score Range,600 < 649,185552000.0,2022-03-31mtg,NaN,NaN,NaN,NaN,False
2,2022-03-31,mtg,Chartered Bank,Uninsured,Credit Score Range,0 < 599,NaN,2022-03-31mtg,NaN,NaN,NaN,NaN,False
3,2022-03-31,mtg,Chartered Bank,Uninsured,Credit Score Range,No Score,NaN,2022-03-31mtg,NaN,NaN,NaN,NaN,False
4,2022-03-31,mtg,Chartered Bank,Uninsured,Credit Score Range,750 or greater,NaN,2022-03-31mtg,NaN,NaN,NaN,NaN,False


In [38]:
with pd.ExcelWriter('Predicted_Losses.xlsx') as writer:
    predicted_loss_df.to_excel(writer, sheet_name='joined_output', index = False)
    per_year.to_excel(writer, sheet_name='Total_Predicted_Losses_Summary')